In [1]:
from Crypto.Cipher import AES

In [2]:
import os

In [3]:
from math import ceil, floor

## Задание 1

In [4]:
aes_block_size = 16
aes_key_allowed_lenghts = (16, 24, 32)

In [5]:
def aes_block_encrypt(key, message, is_final_block=False, padding=0):
    if type(key) is not bytes:
        raise TypeError("Key's type should be bytes.")
    if type(message) is not bytes:
        raise TypeError("Message's type should be bytes.")
    if len(key) not in aes_key_allowed_lenghts:
        raise ValueError("AES key must be either 16, 24, or 32 bytes long")
    if is_final_block:
        message += padding
    if len(message) != aes_block_size:
        raise ValueError("Message's length should be equal to AES block size = 16. Use padding.")
    
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext = cipher.encrypt(message)
    return ciphertext
    
    
def aes_block_decrypt(key, ciphertext):
    if type(key) is not bytes:
        raise TypeError("Key's type should be bytes.")
    if type(ciphertext) is not bytes:
        raise TypeError("Ciphertext's type should be bytes.")
    if len(key) not in aes_key_allowed_lenghts:
        raise ValueError("AES key must be either 16, 24, or 32 bytes long")
    if len(ciphertext) != aes_block_size:
        raise ValueError("Ciphertext's length should be equal to AES block size = 16.")
    
    cipher = AES.new(key, AES.MODE_ECB)
    message = cipher.decrypt(ciphertext)
    return message

## Задание 2

In [6]:
def xor_two_bytes(x, y):
    if len(x) != len(y):
        raise ValueError("Lengths of xored values aren't equal")
    result = bytearray()
    for i in range(len(x)):
        result.append(x[i] ^ y[i])
    return bytes(result)

In [7]:
def pkcs7_padding(data):
    if len(data) % aes_block_size == 0:
        return b''
    padding_len = aes_block_size - len(data) % aes_block_size
    return (chr(padding_len) * padding_len).encode()


def ecb(key, data):
    blocks_amount = ceil(len(data) / aes_block_size)
    ciphertext = b''
    data += pkcs7_padding(data)
    for i in range(blocks_amount):
        ciphertext += aes_block_encrypt(key, data[aes_block_size*i:aes_block_size*(i+1)])
    return ciphertext


def cbc(key, data, iv):
    blocks_amount = ceil(len(data) / aes_block_size)
    ciphertext = b''
    data += pkcs7_padding(data)
    for i in range(blocks_amount):
        plaintext = data[aes_block_size*i:aes_block_size*(i+1)]
        xor_result = xor_two_bytes(plaintext, iv)
        iv = aes_block_encrypt(key, xor_result)
        ciphertext += iv
    return ciphertext


def cfb(key, data, iv):
    blocks_amount = floor(len(data) / aes_block_size)
    ciphertext = b''
    for i in range(blocks_amount):
        aes_result = aes_block_encrypt(key, iv)
        plaintext = data[aes_block_size*i:aes_block_size*(i+1)]
        iv = xor_two_bytes(aes_result, plaintext)
        ciphertext += iv
    last_aes_result = aes_block_encrypt(key, iv)
    last_plaintext = data[aes_block_size*(blocks_amount):]
    ciphertext += xor_two_bytes(last_plaintext, last_aes_result[:len(last_plaintext)])
    return ciphertext


def ofb(key, data, iv):
    blocks_amount = floor(len(data) / aes_block_size)
    ciphertext = b''
    for i in range(blocks_amount):
        plaintext = data[aes_block_size*i:aes_block_size*(i+1)]
        iv = aes_block_encrypt(key, iv)
        ciphertext += xor_two_bytes(iv, plaintext)
    last_plaintext = data[aes_block_size*(blocks_amount):]
    last_aes_result = aes_block_encrypt(key, iv)
    ciphertext += xor_two_bytes(last_plaintext, last_aes_result[:len(last_plaintext)])
    return ciphertext


def ctr(key, data, iv): # iv = nonce + counter
    blocks_amount = ceil(len(data) / aes_block_size)
    ciphertext = b''
    data += pkcs7_padding(data)
    for i in range(blocks_amount):
        aes_result = aes_block_encrypt(key, iv)
        plaintext = data[aes_block_size*i:aes_block_size*(i+1)]
        ciphertext += xor_two_bytes(aes_result, plaintext)
        iv = bytearray(iv)
        iv[-1] += 1
        iv = bytes(iv)
    return ciphertext
        

def aes_encrypt(key, data, mode, iv=b''):
    if mode == 'ECB':
        return ecb(key, data)
    if iv == b'':
        if mode != 'CTR':
            iv = os.urandom(aes_block_size)
        else:
            counter = 1
            iv = os.urandom(aes_block_size // 2) + counter.to_bytes(aes_block_size // 2, 'big')
            return iv + ctr(key, data, iv)
    if mode == 'CBC':
        return iv + cbc(key, data, iv)
    if mode == 'CFB':
        return iv + cfb(key, data, iv)
    if mode == 'OFB':
        return iv + ofb(key, data, iv)
    if mode == 'CTR':
        return iv + ctr(key, data, iv)

## 2.5 - валидация

In [8]:
key = os.urandom(16)
msg1 = os.urandom(16)
msg2 = os.urandom(40)

In [9]:
# ECB

my_ciphertext_ecb1 = aes_encrypt(key, msg1, 'ECB')
my_ciphertext_ecb2 = aes_encrypt(key, msg2, 'ECB')

ecb_cipher = AES.new(key, AES.MODE_ECB)
builtin_ciphertext_ecb1 = ecb_cipher.encrypt(msg1 + pkcs7_padding(msg1))
builtin_ciphertext_ecb2 = ecb_cipher.encrypt(msg2 + pkcs7_padding(msg2))

assert(my_ciphertext_ecb1 == builtin_ciphertext_ecb1)
assert(my_ciphertext_ecb2 == builtin_ciphertext_ecb2)

In [10]:
# CBC
iv = os.urandom(aes_block_size)

my_ciphertext_cbc1 = aes_encrypt(key, msg1, 'CBC', iv)
my_ciphertext_cbc2 = aes_encrypt(key, msg2, 'CBC', iv)

cbc_cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
builtin_ciphertext_cbc1 = cbc_cipher.encrypt(msg1 + pkcs7_padding(msg1))

cbc_cipher2 = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
builtin_ciphertext_cbc2 = cbc_cipher2.encrypt(msg2 + pkcs7_padding(msg2))

assert(my_ciphertext_cbc1[aes_block_size:] == builtin_ciphertext_cbc1)
assert(my_ciphertext_cbc2[aes_block_size:] == builtin_ciphertext_cbc2)

In [11]:
# CFB
iv = os.urandom(aes_block_size)

my_ciphertext_cfb1 = aes_encrypt(key, msg1, 'CFB', iv)
my_ciphertext_cfb2 = aes_encrypt(key, msg2, 'CFB', iv)

cfb_cipher = AES.new(key=key, mode=AES.MODE_CFB, IV=iv, segment_size=128)
builtin_ciphertext_cfb1 = cfb_cipher.encrypt(msg1)

cfb_cipher2 = AES.new(key=key, mode=AES.MODE_CFB, IV=iv, segment_size=128)
builtin_ciphertext_cfb2 = cfb_cipher2.encrypt(msg2 + pkcs7_padding(msg2))

assert(my_ciphertext_cfb1[aes_block_size:] == builtin_ciphertext_cfb1)
assert(my_ciphertext_cfb2[aes_block_size:] == builtin_ciphertext_cfb2[:len(msg2)])

In [12]:
#OFB
iv = os.urandom(aes_block_size)

my_ciphertext_ofb1 = aes_encrypt(key, msg1, 'OFB', iv)
my_ciphertext_ofb2 = aes_encrypt(key, msg2, 'OFB', iv)

ofb_cipher = AES.new(key=key, mode=AES.MODE_OFB, IV=iv)
builtin_ciphertext_ofb1 = ofb_cipher.encrypt(msg1)

ofb_cipher2 = AES.new(key=key, mode=AES.MODE_OFB, IV=iv)
builtin_ciphertext_ofb2 = ofb_cipher2.encrypt(msg2 + pkcs7_padding(msg2))

assert(my_ciphertext_ofb1[aes_block_size:] == builtin_ciphertext_ofb1)
assert(my_ciphertext_ofb2[aes_block_size:] == builtin_ciphertext_ofb2[:len(msg2)])

In [13]:
#CTR
nonce = os.urandom(aes_block_size // 2)
counter = 1

my_ciphertext_ctr1 = aes_encrypt(key, msg1, 'CTR', nonce + counter.to_bytes(aes_block_size // 2, 'big'))
my_ciphertext_ctr2 = aes_encrypt(key, msg2, 'CTR', nonce + counter.to_bytes(aes_block_size // 2, 'big'))

ctr_cipher = AES.new(key=key, mode=AES.MODE_CTR, nonce=nonce, initial_value=1)
builtin_ciphertext_ctr1 = ctr_cipher.encrypt(msg1 + pkcs7_padding(msg1))

ctr_cipher2 = AES.new(key=key, mode=AES.MODE_CTR, nonce=nonce, initial_value=1)
builtin_ciphertext_ctr2 = ctr_cipher2.encrypt(msg2 + pkcs7_padding(msg2))

assert(my_ciphertext_ctr1[aes_block_size:] == builtin_ciphertext_ctr1)
assert(my_ciphertext_ctr2[aes_block_size:] == builtin_ciphertext_ctr2)

## Задание 3

In [14]:
def delete_pkcs7(data):
    possible_padding_element = data[-1]
    if data[-possible_padding_element:] == chr(data[-1]).encode() * possible_padding_element:
        return data[:-possible_padding_element]
    return data


def ecb_decrypt(key, data):
    blocks_amount = ceil(len(data) / aes_block_size)
    plaintext = b''
    for i in range(blocks_amount):
        plaintext += aes_block_decrypt(key, data[aes_block_size*i:aes_block_size*(i+1)])
    plaintext = delete_pkcs7(plaintext)
    return plaintext


def cbc_decrypt(key, ciphertext, iv):
    blocks_amount = ceil(len(ciphertext) / aes_block_size)
    plaintext = b''
    for i in range(blocks_amount):
        ciphertext_block = ciphertext[aes_block_size*i:aes_block_size*(i+1)]
        aes_decrtypt_result = aes_block_decrypt(key, ciphertext_block)
        plaintext += xor_two_bytes(aes_decrtypt_result, iv)
        iv = ciphertext_block
    plaintext = delete_pkcs7(plaintext)
    return plaintext


def cfb_decrypt(key, ciphertext, iv):
    blocks_amount = floor(len(ciphertext) / aes_block_size)
    plaintext = b''
    for i in range(blocks_amount):
        aes_result = aes_block_encrypt(key, iv)
        iv = ciphertext[aes_block_size*i:aes_block_size*(i+1)]
        plaintext += xor_two_bytes(iv, aes_result)
    last_aes_result = aes_block_encrypt(key, iv)
    last_ciphertext_block = ciphertext[aes_block_size*(blocks_amount):]
    plaintext += xor_two_bytes(last_ciphertext_block, last_aes_result[:len(last_ciphertext_block)])
    return plaintext


def ctr_decrypt(key, data, iv): # iv = nonce + counter
    blocks_amount = ceil(len(data) / aes_block_size)
    plaintext = b''
    for i in range(blocks_amount):
        aes_result = aes_block_encrypt(key, iv)
        ciphertext_block = data[aes_block_size*i:aes_block_size*(i+1)]
        plaintext += xor_two_bytes(aes_result[:len(ciphertext_block)], ciphertext_block)
        iv = bytearray(iv)
        iv[-1] += 1
        iv = bytes(iv)
    plaintext = delete_pkcs7(plaintext)
    return plaintext


def decrypt(key, ciphertext, mode):
    if mode == 'ECB':
        return ecb_decrypt(key, ciphertext)
    iv = ciphertext[:aes_block_size]
    ciphertext = ciphertext[aes_block_size:]
    if mode == 'CBC':
        return cbc_decrypt(key, ciphertext, iv)
    if mode == 'CFB':
        return cfb_decrypt(key, ciphertext, iv)
    if mode == 'OFB':
        return ofb(key, ciphertext, iv)
    if mode == 'CTR':
        return ctr_decrypt(key, ciphertext, iv)

In [15]:
cbc_key = bytes(bytearray.fromhex('140b41b22a29beb4061bda66b6747e14'))
cbc_ciphertext_1 = bytes(bytearray.fromhex('4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81'))
cbc_ciphertext_2 = bytes(bytearray.fromhex('5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253'))

print('CBC message 1 - ', decrypt(cbc_key, cbc_ciphertext_1, 'CBC'))
print('CBC message 2 - ', decrypt(cbc_key, cbc_ciphertext_2, 'CBC'))

CBC message 1 -  b'Basic CBC mode encryption needs padding.'
CBC message 2 -  b'Our implementation uses rand. IV'


In [16]:
ctr_key = bytes(bytearray.fromhex('36f18357be4dbd77f050515c73fcf9f2'))
ctr_ciphertext_1 = bytes(bytearray.fromhex('69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329'))
ctr_ciphertext_2 = bytes(bytearray.fromhex('770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451'))

print('CTR message 1 - ', decrypt(ctr_key, ctr_ciphertext_1, 'CTR'))
print('CTR message 2 - ', decrypt(ctr_key, ctr_ciphertext_2, 'CTR'))

CTR message 1 -  b'CTR mode lets you build a stream cipher from a block cipher.'
CTR message 2 -  b'Always avoid the two time pad!'


## Задание 4

In [17]:
my_secure_message = b"it's-my-secure-message-with-40-bytes-len"

In [18]:
# ECB
ecb_test_encrypt = aes_encrypt(key, my_secure_message, 'ECB')
print(ecb_test_encrypt, 'Len: %i Bytes\n' % len(ecb_test_encrypt))
print(decrypt(key, ecb_test_encrypt, 'ECB'))

b'\xeb4\xfb\t\x97\xd0\n\rB\x15\xb0\xc2\xde\xac09\x1d\x04\x98\x02\xff\x8f@\x0b\xb4\x84\xde*|\x08\x1d\xcd\x08\x8cW\xfa&\xe6V\x04\xa8\xfc\xaa\xa4=G\xc4U' Len: 48 Bytes

b"it's-my-secure-message-with-40-bytes-len"


In [19]:
# CBC
iv_cbc = os.urandom(aes_block_size)
cbc_test_encrypt = aes_encrypt(key, my_secure_message, 'CBC')
print(cbc_test_encrypt, 'Len: %i Bytes\n' % len(cbc_test_encrypt))
print(decrypt(key, cbc_test_encrypt, 'CBC'))

b'\x04t\xbd\xf2Q\x8b\xbe{p:\x00\xe0h$\x07\xd4\x87\xd6I#\x82S\xad\x9c\xdc-%t\n<\x85\t;oAr\x9d\x9f?\xea%Q:\xd5\x13\xa5\x7f VWFK\xd4\x1e%\xc6\x17p7\xe3\xc2\xf9\xdd\x96' Len: 64 Bytes

b"it's-my-secure-message-with-40-bytes-len"


In [20]:
# CFB
iv_cfb = os.urandom(aes_block_size)
cfb_test_encrypt = aes_encrypt(key, my_secure_message, 'CFB')
print(cfb_test_encrypt, 'Len: %i Bytes\n' % len(cfb_test_encrypt))
print(decrypt(key, cfb_test_encrypt, 'CFB'))

b'\x9c@Jv\xe6J\xccF\rZ\xf8\xc4\xde\xb1\xec\x17*\xaa\xafk\xael <\xc8\x96\x08\xbe\xa1\xc8\xe3\x9a\x1a\xaa\xb7\xe1\xb4\x10\x99\xc72\xe7\x7f.\x94\x04mYU\xb8\x9e\x9b{d2\xf9' Len: 56 Bytes

b"it's-my-secure-message-with-40-bytes-len"


In [21]:
# OFB
iv_ofb = os.urandom(aes_block_size)
ofb_test_encrypt = aes_encrypt(key, my_secure_message, 'OFB')
print(ofb_test_encrypt, 'Len: %i Bytes\n' % len(ofb_test_encrypt))
print(decrypt(key, ofb_test_encrypt, 'OFB'))

b"\xe5\xc5\xea\xb7\xa3\xd4\xc1J\xaa\xe6)'t\x00Ro\x81\x8fEG\x1e}\xbd\xed\xda\xc3\xae\x97pc\x18\xdf\xd8\x93\xd9\x13\x8e\x11N\x06!r\x89,\xc8\x05%\xde\x1f\x80\xe6\xb1#\x10\xc0\xff" Len: 56 Bytes

b"it's-my-secure-message-with-40-bytes-len"


In [22]:
# CTR
nonce = os.urandom(aes_block_size // 2)
counter = 1
ctr_test_encrypt = aes_encrypt(key, my_secure_message, 'CTR', nonce + counter.to_bytes(aes_block_size // 2, 'big'))
print(ctr_test_encrypt, 'Len: %i Bytes\n' % len(ctr_test_encrypt))
print(decrypt(key, ctr_test_encrypt, 'CTR'))

b'N\xef\xd3\xefD\xb7\xdc\xfa\x00\x00\x00\x00\x00\x00\x00\x01\xd2\x7f\x9c@\xb1\x18^\x97C\x08\x8f\xbb\xd6\xd5\xe7\x80\xbeW\x86\xa3\xe4p\x13l(\xcdEj\xb4\x91\x1b\xacD\xde\x92\xbd\xfa\x9e\xf94\xb09\x02\xcf\x95\xf7|\x17' Len: 64 Bytes

b"it's-my-secure-message-with-40-bytes-len"
